<a href="https://colab.research.google.com/github/biranchi2018/My-NLP-Examples/blob/master/17.MultiLabel_Classification_using_SimpleTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SimpleTransformers

     |████████████████████████████████| 194kB 3.5MB/s 
     |████████████████████████████████| 675kB 10.7MB/s 
     |████████████████████████████████| 3.8MB 10.2MB/s 
     |████████████████████████████████| 204kB 31.8MB/s 
     |████████████████████████████████| 1.1MB 45.8MB/s 
     |████████████████████████████████| 890kB 42.2MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=cd82a355a8e5ad655a4601c300915477bb7ecd65647a0f51469b11f9863146c8
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b8b458160e531bdd488cc3cd79c782dae0d0c5916d5992258576ebba98ef9be1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [0]:
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel

In [3]:
# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns, a 'text' and a 'labels' column. The `labels` column should contain multi-hot encoded lists.
train_data = [["Example sentence 1 for multilabel classification.", [1, 1, 1, 1, 0, 1]]] + [
    ["This is another example sentence. ", [0, 1, 1, 0, 0, 0]]
]
train_data

[['Example sentence 1 for multilabel classification.', [1, 1, 1, 1, 0, 1]],
 ['This is another example sentence. ', [0, 1, 1, 0, 0, 0]]]

In [4]:
train_df = pd.DataFrame(train_data, columns=["text", "labels"])
train_df.head()

,text,labels
0,Example sentence 1 for multilabel classification.,"[1, 1, 1, 1, 0, 1]"
1,This is another example sentence.,"[0, 1, 1, 0, 0, 0]"


In [5]:
eval_data = [
    ["Example eval sentence for multilabel classification.", [1, 1, 1, 1, 0, 1]],
    ["Example eval sentence belonging to class 2", [0, 1, 1, 0, 0, 0]],
]
eval_df = pd.DataFrame(eval_data)
eval_df.head()

,0,1
0,Example eval sentence for multilabel classific...,"[1, 1, 1, 1, 0, 1]"
1,Example eval sentence belonging to class 2,"[0, 1, 1, 0, 0, 0]"


In [6]:
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=6,
    use_cuda=False,
    args={"reprocess_input_data": True, "overwrite_output_dir": True, "num_train_epochs": 5},
)
# You can set class weights by using the optional weight argument
print(train_df.head())


                                                text              labels
0  Example sentence 1 for multilabel classification.  [1, 1, 1, 1, 0, 1]
1                 This is another example sentence.   [0, 1, 1, 0, 0, 0]


In [7]:
# Train the model
model.train_model(train_df)

Running loss: 0.788232


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.678667


Running loss: 0.667204


Running loss: 0.661074


Running loss: 0.648439



In [8]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(f'result: {result}')
print(f'model_outputs: {model_outputs}')

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:683: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



result: {'LRAP': 0.7916666666666666, 'eval_loss': 0.6505746245384216}
model_outputs: [[0.566651   0.55958873 0.5637633  0.49448508 0.48484018 0.5091752 ]
 [0.553328   0.5512762  0.54670733 0.49007237 0.49228498 0.49647245]]


In [11]:
predictions, raw_outputs = model.predict(["This thing is entirely different from the other thing."])
print(f'predictions: {predictions}')
print(f'raw_outputs: {raw_outputs}')


predictions: [[1, 1, 1, 0, 0, 1]]
raw_outputs: [[0.5640721  0.55558485 0.55666965 0.49613094 0.48839578 0.5012629 ]]
